<center><img src="https://mlflow.org/docs/0.4.1/_static/MLflow-logo-final-black.png" width="300" align="center" /><center>
 <center><h1><i>“platform for the machine learning lifecycle”</i></h1><center>
MLFlow es una plataforma open source que permite administrar el ciclo de vida de ML, incluyendo la experimentación, reproducibilidad y deploy de modelos, además de brindar la posibilidad de comparar la performance de los modelos obtenidos a lo largo del proyecto. Es posible utilizarla con múltiples lenguajes, tanto de manera local como en la nube. 

## Ejemplo
#### Iris DataSet <br>
Dataset = Un conjunto de datos o dataset corresponde a los contenidos de una única tabla de base de datos o una única matriz de datos de estadística, donde cada columna de la tabla representa una variable en particular, y cada fila representa a un miembro determinado del conjunto de datos que estamos tratando.<br>
Dataset Iris = Contiene datos para cuantificar la variación morfológica de la flor Iris de tres especies relacionadas. <br>
Columnas:
* Largo de sépalo
* Ancho de sépalo
* Largo de pétalo
* Ancho de pétalo
* Especies <- lo que queremos predecir


<center><img src="https://www.w3resource.com/w3r_images/iris_flower_dataset.png" width="600" align="center" /><center>

<center><img src="https://miro.medium.com/max/1100/0*SHhnoaaIm36pc1bd" width="700" align="center" /><center>
<br>



# Ejemplo de implementación: <br> Usando el algoritmo KNeighborsClassifier de scikit-learn 

## Primer Paso
Instalar la libreria **mlflow**  con `pip install mlflow`

In [ ]:
!pip install mlflow

Importar librerias

In [ ]:
import numpy as np 
import pandas as pd # procesamiento de datos
import mlflow
import mlflow.sklearn # como voy a entrenar un modelo de sklearn, debo importar este módulo de mlflow
from sklearn import datasets # para importar el dataset iris directamente desde los dataset que provee sklearn

In [ ]:
iris_X, iris_y = datasets.load_iris(return_X_y=True)
np.unique(iris_y)
np.random.seed(0)
indices = np.random.permutation(len(iris_X))
iris_X_train = iris_X[indices[:-10]]
iris_y_train = iris_y[indices[:-10]]
iris_X_test = iris_X[indices[-10:]]
iris_y_test = iris_y[indices[-10:]]

In [ ]:

# Create and fit a nearest-neighbor classifier
from sklearn.neighbors import KNeighborsClassifier   

n = 5

#mlflow.log_param('n_neighbors', n) # Guardar parametro "n" correspondiente al hyperparametro n_neighbors de los KNN

knc = KNeighborsClassifier(n_neighbors = n)
knc.fit(iris_X_train, iris_y_train)


knc.predict(iris_X_test)
score = knc.score(iris_X_test, iris_y_test)


print('Metrica: {}'.format(score))
print('Predicción: {}'.format(knc.predict(iris_X_test)))
print('Valor Real: {}'.format(iris_y_test))

In [ ]:
mlflow.start_run()

In [ ]:
# Create and fit a nearest-neighbor classifier
from sklearn.ensemble import RandomForestClassifier

n_arboles = 50
profundidad = 10

rf = RandomForestClassifier(n_estimators = n_arboles, max_depth = profundidad)
rf.fit(iris_X_train,iris_y_train)

mlflow.log_param('_tipo_algoritmo','RandomForestClassifier')
mlflow.log_params( rf.get_params())

score = rf.score(iris_X_test, iris_y_test)

mlflow.log_metric('score_iris', score)
mlflow.sklearn.log_model(rf, 'model')

print('Metrica: {}'.format(score))
print('Predicción: {}'.format(rf.predict(iris_X_test)))
print('Valor Real: {}'.format(iris_y_test))

In [ ]:
mlflow.end_run()

In [ ]:
with mlflow.start_run():
    # Create and fit a nearest-neighbor classifier
    from sklearn.neighbors import KNeighborsClassifier   
    
    n = 10
    
    #mlflow.log_param('n_neighbors', n) # Guardar parametro "n" correspondiente al hyperparametro n_neighbors de los KNN
    
    knc = KNeighborsClassifier(n_neighbors = n)
    knc.fit(iris_X_train, iris_y_train)
    
    mlflow.log_param('_tipo_algoritmo','KNeighborsClassifier')
    mlflow.log_params(knc.get_params())
    
    knc.predict(iris_X_test)
    score = knc.score(iris_X_test, iris_y_test)
    
    mlflow.log_metric('score_iris', score) # Guardar la métrica score obtenido de knn
    mlflow.sklearn.log_model(knc, 'model') # Guardar la métrica score obtenido de knn
    
    print('Metrica: {}'.format(score))
    print('Predicción: {}'.format(knc.predict(iris_X_test)))
    print('Valor Real: {}'.format(iris_y_test))

### Explorar los resultados obtenidos
Podemos ver los resultados que fuimos guardando en diferentes corridas a partir del comando `search_runs`.
Primero podemos consultar los experimentos disponibles usando el comando: `mlflow.tracking.MlflowClient().list_experiments()`


In [ ]:
mlflow.tracking.MlflowClient().list_experiments()

In [ ]:
mlflow.search_runs().head(5)

### Formas de filtrar los runs
Hay dos clases para los comparadores: numericos y string.<br>

__Comparadores númericos__  (metrics): =, !=, >, >=, <, and <=.<br>
__Comparadores para string__  (params, tags, and attributes): = and !=.<br>

##### Ejemplos:
Para buscar el subconjunto de ejecuciones con una métrica de precisión registrada mayor que 0.92:<br>
`metrics.accuracy > 0.92`

Para buscar ejecuciones creadas utilizando un modelo de Regresión logística, una tasa de aprendizaje (lambda) de 0.001 y una métrica de error registrada por debajo de 0.05:<br>
`params.model = "LogisticRegression" and params.lambda = "0.001" and metrics.error <= 0.05`

Para buscar los runs que han fallado en su ejecución:<br>
`attributes.status = "FAILED"`

In [ ]:
mlflow.search_runs(experiment_ids='0', filter_string= 'metrics.score_iris > 0.9').head()

### Cargar un modelo guardado

In [ ]:
type(mlflow.search_runs())

Obtener direccion del run del cual nos interesa recuperar el modelo guardado

In [ ]:
model_uri = mlflow.search_runs()\
        [mlflow.search_runs()['run_id'] == '5ce2a95886874e9fac49da899ecee6f5']\
        .artifact_uri.item()
model_uri

In [ ]:
model = mlflow.sklearn.load_model(model_uri+'/model')

In [ ]:
type(model)

In [ ]:
model= mlflow.sklearn.load_model(model_uri+'/model')
print('Predicción: {}'.format(model.predict(iris_X_test)))
print('Valor Real: {}'.format(iris_y_test))

In [ ]:
df = pd.DataFrame([model.predict(iris_X_test),iris_y_test]).transpose().rename({0:"predicho", 1:"valor_real"}, axis = 1)
iris_class = {0:'Iris-Setosa',1:'Iris-Versicolour',2:'Iris-Virginica'}
df.replace(iris_class, inplace = True)
df['son_iguales'] = df.apply(lambda x: True if x.predicho == x.valor_real else False, axis = 1)

In [ ]:
df

### Interfaz
Tambien contamos con una interfaz gráfica para el analisis de los resultados de los modelo.
Podemos acceder escribiendo en la consola de anaconda **mlflow ui** parados en el directorio donde se encuentran almacenados los runs de mlflow. Podemos ver la interfaz accediendo a http://localhost:5000/ en nuestro navegador.
<center><img src="https://miro.medium.com/max/4872/1*Swexh591ukDYIWNhTQ7YNA.png" width="800" align="center" /><center>

